In [1]:
import pandas
from desi_findspec import find_desi_spec

# Make a pandas dataframe that has the RA and DECs that you want

data = pandas.DataFrame( [ {'name': 'SN09alu', 'ra': 214.572816, 'dec': 53.788987 },
                           {'name': 'SN09a', 'ra': 215.982879, 'dec': 35.185546 },
                           {'name': 'SN09ac', 'ra': 186.14969, 'dec': 47.237527 } ] )

# Pass this dataframe to find_desi_spec

desipasswd = "putdespasswdhere"
result = find_desi_spec( data, radius=1./3600., release='daily', desipasswd=desipasswd )

[2022-05-20 08:56:52 - Have searched {i} of {len(df)} positions for DESI spectra...


In [7]:
# Notice that there are multiple rows where multiple spectra were found.
# The rows where nothing was found are NaN
result

,name,ra,dec,targetid,tile,petal,night,z,zerr,zwarn,filename,spectype,subtype
0,SN09alu,214.572816,53.788987,3.963330e+16,174.0,4.0,20210405.0,0.074413,0.000028,0.0,daily/tiles/cumulative/174/20210405/zbest-4-17...,GALAXY,
0,SN09alu,214.572816,53.788987,3.963330e+16,182.0,4.0,20210506.0,0.074455,0.000025,0.0,daily/tiles/cumulative/182/20210506/zbest-4-18...,GALAXY,
1,SN09a,215.982879,35.185546,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SN09ac,186.149690,47.237527,3.963320e+16,20620.0,9.0,20220421.0,0.163469,0.000044,0.0,daily/tiles/cumulative/20620/20220421/redrock-...,GALAXY,


In [8]:
# You can pull out the ones that had observations with notnull():
result[ result.targetid.notnull() ]

,name,ra,dec,targetid,tile,petal,night,z,zerr,zwarn,filename,spectype,subtype
0,SN09alu,214.572816,53.788987,3.963330e+16,174.0,4.0,20210405.0,0.074413,0.000028,0.0,daily/tiles/cumulative/174/20210405/zbest-4-17...,GALAXY,
0,SN09alu,214.572816,53.788987,3.963330e+16,182.0,4.0,20210506.0,0.074455,0.000025,0.0,daily/tiles/cumulative/182/20210506/zbest-4-18...,GALAXY,
2,SN09ac,186.149690,47.237527,3.963320e+16,20620.0,9.0,20220421.0,0.163469,0.000044,0.0,daily/tiles/cumulative/20620/20220421/redrock-...,GALAXY,


In [9]:
# You can pull out the ones without observations with isnull()
result[ result.targetid.isnull() ]

,name,ra,dec,targetid,tile,petal,night,z,zerr,zwarn,filename,spectype,subtype
1,SN09a,215.982879,35.185546,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
print( f'{len(result[result.targetid.notnull()].groupby("name"))} have spectra' )
print( f'{len(result[result.targetid.isnull()].groupby("name"))} do not' ) 

2 have spectra
1 do not
